In [138]:
# This class ensures that the FSM loaded from the CSV file has a correct amount of transitions.
class FSMFileTransitionsError(Exception):
    
    def __init__(self, list_of_states):
        print("States of the following unique IDs have an amount of 'possible discrete actions (transitions)' that is not equal to the amount of transitions in their 'Transition names' field.")
        print(*list_of_states)
        print("Please recheck the states and try again.")

In [139]:
# This class ensures that the FSM loaded from the CSV file has a correct amount of transitions.
class FSMStartStateNotFoundError(Exception):
    
    def __init__(self):
        print("No start states have been found in the CSV file!")
        print("Please recheck the file and set at least a single start state.")

In [140]:
class FSMTransitionStateNotFoundError(Exception):

    def __init__(self, unique_id):
        print("The transition to a state of the unique ID: {} has failed due to the state not being found in the CSV file!".format(unique_id))
        print("If this was not expected, please check the CSV file.")

In [141]:
class FSMIncorrectPresetStartStateError(Exception):

    def __init__(self, unique_id):
        print("The provided preset start state of the unique ID {} has not been found, or is not set as a start state.".format(unique_id))
        print("Please change the preset ID or set the state to be a start state in the CSV file.")

In [142]:
import pandas

def convert_state_names(file_name):
    df = pandas.read_csv(file_name)

    if 'Discretized_state_name' not in df.columns:
        state_names_dict = {}
        counter = 1
        for row in df.itertuples(index=False):
            print(row)
            if row.State_name not in state_names_dict:
                state_names_dict[row.State_name] = counter
                print(state_names_dict[row.State_name])
                counter += 1
        
        df['Discretized_state_name'] = df['State_name'].map(state_names_dict)
        df.to_csv(file_name, index=False)

    else:
        print('INFO: The file provided already contains discretized state names.')

In [143]:
convert_state_names('pelican.csv')

INFO: The file provided already contains discretized state names.


In [144]:
import numpy as np
import gym
from stable_baselines3.common.env_checker import check_env
from csv import DictReader

class FSMEnv(gym.Env):
  """
  Custom Environment that follows gym interface.
  This is a simple env where the agent must learn how to traverse through a FSM in a CSV file. 
  """
  # If using google colab, GUI cannot be implemented ('human' render mode)
  # metadata = {'render.modes': ['console']}

  def __init__(self, file_name, start_state_id=None):
    super(FSMEnv, self).__init__()

    # The name of the .csv file containing the FSM
    self.file_name = file_name
    self.start_state_id = start_state_id

    convert_state_names(file_name)

    if start_state_id is None:
      preset_start_state = False
    else:
      preset_start_state = True
      self.start_state_id = int(start_state_id)

      with open(self.file_name, 'r') as csv_file:
        correct_preset_start_state = False
        # pass the file object to DictReader() to get the DictReader object
        csv_dict_reader = DictReader(csv_file)

        # iterate over each line as a ordered dictionary
        for row in csv_dict_reader:
          # row variable is a dictionary that represents a row in csv
          if int(row['Unique_ID']) == self.start_state_id:
            if row['Start_state'] == '1':
              correct_preset_start_state = True
        
        if correct_preset_start_state == False:
          raise FSMIncorrectPresetStartStateError(self.start_state_id)

    if preset_start_state == False:  
      start_state_already_set = False
      multiple_start_states = False

      with open(self.file_name, 'r') as csv_file:
        # pass the file object to DictReader() to get the DictReader object
        csv_dict_reader = DictReader(csv_file)
        start_states = []

        # iterate over each line as a ordered dictionary
        for row in csv_dict_reader:
          # row variable is a dictionary that represents a row in csv
          if row['Start_state'] == '1':
            start_states.append(int(row['Unique_ID']))
            
            if start_state_already_set == False:
              start_state_already_set = True
              self.start_state_id = int(row['Unique_ID'])
            else:
              self.start_state_id = None
              multiple_start_states = True
      
        if multiple_start_states == True:
          print("Multiple start states have been found!")
          correct_start_state_provided = False

          while correct_start_state_provided == False:
            print("Please choose one of the found start states:")
            for unique_id in start_states:
              print(unique_id)

            self.start_state_id = int(input("Provide an unique ID: "))
            
            if self.start_state_id not in start_states:
              print("ID provided is not in the list!")
            
            else:
              correct_start_state_provided = True
        
        if start_state_already_set == False:
          raise FSMStartStateNotFoundError()
        
    row_counter = 0

    # Define action and observation space
    # They must be gym.spaces objects
    # Example when using discrete actions, we have two: left and right
    with open(self.file_name, 'r') as csv_file:
      csv_dict_reader = DictReader(csv_file)
      broken_states = []
      broken_states_exist = False

      for row in csv_dict_reader:
        row_counter += 1

        if len(row['Transitions_to_states'].split()) != int(row['Possible_discrete_actions_(transitions)']):
          broken_states_exist = True
          broken_states.append(row['Unique_ID'])

        if int(row['Unique_ID']) == self.start_state_id:
          initial_actions = int(row['Possible_discrete_actions_(transitions)'])
          self.action_space = gym.spaces.Discrete(initial_actions)
    
    if broken_states_exist == True:
      raise FSMFileTransitionsError(broken_states)

    self.past_discretized_states = np.zeros((100,), dtype=np.uint64)
    self.past_unique_id_states = np.zeros((100,), dtype=np.uint64)
    self.amount_of_states_visited = 0 # Counter for the past 2 arrays as well
    self.current_unique_id_state = self.start_state_id
    self.current_discretized_state = 0

    # Dict observation space
    self.observation_space = gym.spaces.Dict(
    {
        # Current state obs
        'current_state': gym.spaces.Discrete(row_counter),
        # Past states history obs
        'past_states': gym.spaces.Box(low=0, high=np.inf, shape=(100,), dtype=np.uint64),
        # Transitions to states ?
        # 'transitions_to_states': spaces.Box(low=0, high=row_counter, shape=(100,), dtype=np.uint8),
        # Amount of states already visited (could be garbage data?)
        'amount_of_states': gym.spaces.Discrete(row_counter),
    })

    print(self.action_space)
    print(self.observation_space)
    print("Amount of rows: {}".format(row_counter))

  def reset(self):
    """
    Important: the observation must be a numpy array
    :return: (np.array) 
    """
    # Initialize the agent at the start state
    self.past_discretized_states = np.zeros((100,), dtype=np.uint64)
    self.past_unique_id_states = np.zeros((100,), dtype=np.uint64)
    self.past_states = np.zeros((100,), dtype=int)
    self.amount_of_states_visited = 0
    self.agent_pos = self.start_state_id

    with open(self.file_name, 'r') as csv_file:
      csv_dict_reader = DictReader(csv_file)
      
      for row in csv_dict_reader:
        if self.start_state_id == int(row['Unique_ID']):
          self.current_discretized_state = int(row['Discretized_state_name'])
          self.past_discretized_states[self.amount_of_states_visited] = int(row['Discretized_state_name'])
          self.past_unique_id_states[self.amount_of_states_visited] = int(row['Unique_ID'])
          # self.amount_of_states_visited = 1
          # Leaving it at 0 will be a bit untrue, but better for the reward.

    observation = {
      'current_state': self.current_discretized_state,
      'past_states': self.past_discretized_states,
      'amount_of_states': self.amount_of_states_visited, 
    }

    return observation

  def step(self, action):
    # NEED TO STORE UNIQUE IDs AS A SEPARATE LIST (NOT OBS)
    # this is done tbh
    with open(self.file_name, 'r') as csv_file:
      csv_dict_reader = DictReader(csv_file)

      for row in csv_dict_reader:
        if int(row['Discretized_state_name']) == self.current_discretized_state:
          # The line below splits the string into a list that contains the unique IDs of states that are possible to transition to
          # from the current state, and uses the action discrete integer to choose a transition from the list.
          # The int() function just converts the string to an integer.
          self.current_unique_id_state = int(row['Transitions_to_states'].split()[action])
      
      new_state_successfully_found = False

      # NEED TO IMPLEMENT CURRENT_UNIQUE_ID_STATE
      # done as well ayyyy
      with open(self.file_name, 'r') as csv_file:
        csv_dict_reader = DictReader(csv_file)
        
        for row in csv_dict_reader:
          if int(row['Unique_ID']) == self.current_unique_id_state:
            new_state_successfully_found = True
            self.current_discretized_state = int(row['Discretized_state_name'])

            if self.current_unique_id_state in self.past_unique_id_states:
              reward = 0
              done = True

            else:
              reward = self.amount_of_states_visited + 1
              done = False

            self.past_discretized_states[self.amount_of_states_visited] = int(row['Discretized_state_name'])
            self.past_unique_id_states[self.amount_of_states_visited] = int(row['Unique_ID'])
            
            actions = int(row['Possible_discrete_actions_(transitions)'])
            self.action_space = gym.spaces.Discrete(actions)
            
            self.amount_of_states_visited += 1
        
      if new_state_successfully_found == False:
        raise FSMTransitionStateNotFoundError(self.current_unique_id_state)

    info = {}

    observation = {
      'current_state': self.current_discretized_state,
      'past_states': self.past_discretized_states,
      'amount_of_states': self.amount_of_states_visited, 
    }

    print(self.current_unique_id_state, action, done, reward, self.action_space)
    return observation, reward, done, info

  def render(self, mode='console'):
    ### Need to implement this now - matplotlib with circles?
    if mode != 'console':
      raise NotImplementedError()
    # agent is represented as a cross, rest as a dot
    print("." * self.agent_pos, end="")
    print("x", end="")
    print("." * (self.grid_size - self.agent_pos))

  def close(self):
    pass

In [145]:
env = FSMEnv('pelican.csv')
# If the environment don't follow the interface, an error will be thrown
check_env(env, warn=True)
#print(env.start_state_id)

INFO: The file provided already contains discretized state names.
Multiple start states have been found!
Please choose one of the found start states:
1
2
4
6
Dupa
Discrete(2)
Dict(amount_of_states:Discrete(6), current_state:Discrete(6), past_states:Box(0, 9223372036854775808, (100,), uint64))
Amount of rows: 6
5 0 False 1 Discrete(2)


In [146]:
env = FSMEnv('pelican.csv')
env.reset()
for _ in range(50):
    env.step(env.action_space.sample()) # take a random action
env.close()

INFO: The file provided already contains discretized state names.
Multiple start states have been found!
Please choose one of the found start states:
1
2
4
6
Dupa
Discrete(2)
Dict(amount_of_states:Discrete(6), current_state:Discrete(6), past_states:Box(0, 9223372036854775808, (100,), uint64))
Amount of rows: 6
5 0 False 1 Discrete(2)
6 0 False 2 Discrete(2)
2 1 False 3 Discrete(2)
5 0 True 0 Discrete(2)
4 1 False 5 Discrete(2)
5 0 True 0 Discrete(2)
4 1 True 0 Discrete(2)
5 0 True 0 Discrete(2)
6 0 True 0 Discrete(2)
2 1 True 0 Discrete(2)
3 1 False 11 Discrete(2)
6 0 True 0 Discrete(2)
2 1 True 0 Discrete(2)
3 1 True 0 Discrete(2)
4 1 True 0 Discrete(2)
3 1 True 0 Discrete(2)
4 1 True 0 Discrete(2)
5 0 True 0 Discrete(2)
6 0 True 0 Discrete(2)
2 1 True 0 Discrete(2)
5 0 True 0 Discrete(2)
6 0 True 0 Discrete(2)
2 1 True 0 Discrete(2)
5 0 True 0 Discrete(2)
6 0 True 0 Discrete(2)
2 1 True 0 Discrete(2)
5 0 True 0 Discrete(2)
4 1 True 0 Discrete(2)
5 0 True 0 Discrete(2)
4 1 True 0 Disc

In [ ]:
import gym

from stable_baselines3 import PPO

env = gym.make('CartPole-v0')

model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)

obs = env.reset()
for i in range(1000):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
      obs = env.reset()

In [ ]:
env = GoLeftEnv(grid_size=10)

obs = env.reset()
env.render()

print(env.observation_space)
print(env.action_space)
print(env.action_space.sample())

GO_LEFT = 0
# Hardcoded best agent: always go left!
n_steps = 20
for step in range(n_steps):
  print("Step {}".format(step + 1))
  obs, reward, done, info = env.step(GO_LEFT)
  print('obs=', obs, 'reward=', reward, 'done=', done)
  env.render()
  if done:
    print("Goal reached!", "reward=", reward)
    break

In [ ]:
from stable_baselines3 import DQN, PPO, A2C
from stable_baselines3.common.cmd_util import make_vec_env

# Instantiate the env
env = GoLeftEnv(grid_size=10)
# wrap it
env = make_vec_env(lambda: env, n_envs=1)

In [ ]:
# Train the agent
model = PPO('MlpPolicy', env, verbose=1).learn(5000)

In [ ]:
# Test the trained agent
obs = env.reset()
n_steps = 20
for step in range(n_steps):
  action, _ = model.predict(obs, deterministic=True)
  print("Step {}".format(step + 1))
  print("Action: ", action)
  obs, reward, done, info = env.step(action)
  print('obs=', obs, 'reward=', reward, 'done=', done)
  env.render(mode='console')
  if done:
    # Note that the VecEnv resets automatically
    # when a done signal is encountered
    print("Goal reached!", "reward=", reward)
    break